Basado en el scrapper hecho por México Abierto:
https://github.com/mxabierto/scraper-sniim/blob/master/sniim/precios_historicos.py


In [1]:
# Importar librerías

import re
import sys
import csv
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import HTTPError
import logging

import os, shutil
from datetime import datetime, timedelta

In [21]:
#shutil.rmtree("raw_data")

In [2]:
# Creación de carpetas temporales y de salida
#print(os.getcwd())
subdir1 = 'temp'
subdir2 = 'raw_data'

try:
  if not os.path.exists(subdir1):
    os.makedirs(subdir1)
  if not os.path.exists(subdir2):
    os.makedirs(subdir2)
except Exception as e:
        print ("Error al crear carpeta: ", e)


C:\Users\Janneth\Documents\MCD\Semestre1\Ing. de Características\RedBAMX\descarga


In [3]:
# Función que realiza el scrapping para extraer una tabla de información y  guardarla en archivo CSV
def creaTabla(url, archivo_salida, paginacion, categoria):
    print (url, archivo_salida, paginacion, categoria)
    try:

        with urllib.request.urlopen(url+str(paginacion), timeout=500) as response, \
          open('temp/'+archivo_salida, 'wb') as out_file, \
          open('raw_data/'+archivo_salida+".csv", 'w', newline="\n") as csvfile:

            data = response.read().decode('utf-8').encode('utf-8')
            soup = BeautifulSoup(data)

            #print(soup.find('span', attrs={"id":"lblPaginacion"}))
            #print(soup.find('span', attrs={"id":"lblPaginacion"}).get_text())

            try:
              #print(soup.find('span', attrs={"id":"lblPaginacion"}).get_text())
              paginas = soup.find('span', attrs={"id":"lblPaginacion"}).get_text()
              #print(paginas)
              print(paginas, paginas != 'Página  1 de  1')



              total_paginas = paginas.split(' ')[-1]
              #print(total_paginas)
              if total_paginas != '1':
                  #print("totalpag dif 1: ", paginacion * int(total_paginas))
                  return creaTabla(url, archivo_salida, paginacion * int(total_paginas), categoria)
              out_file.write(data)


              table = soup.find('table',attrs={"id":"tblResultados"})

              spamwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

              for row in table.find_all("tr"):
                  x = [td.get_text() for td in row.find_all("td")]
                  #x.append(categoria)
                  #print(len(x))
                  #print(x)
                  spamwriter.writerow(x)



            except:
              print ('La página no tiene tabla, ', url, paginacion)
              #borrar archivos
              if os.path.exists("raw_data/"+archivo_salida+".csv"):
                os.remove("raw_data/"+archivo_salida+".csv")
              if os.path.exists("temp/"+archivo_salida):
                os.remove("temp/"+archivo_salida)
              pass
    except Exception as e:
      print ("Error: ", url,e)



In [7]:
# Se definen las fechas de búsqueda
str_fecha_inicio = "01/01/2020"
str_fecha_final = "30/12/2023"
max_intervalo_años = 5

# Se convierten a tipo date
dt_fecha_inicio = datetime.strptime(str_fecha_inicio, '%d/%m/%Y')
dt_fecha_final = datetime.strptime(str_fecha_final, '%d/%m/%Y')

# Ligas a precios
#"http://www.economia-sniim.gob.mx/nuevo/mapa.asp"
#"http://www.economia-sniim.gob.mx/e_MenNal.asp" 
mapa_precios_url = "http://www.economia-sniim.gob.mx/e_MenNal.asp"
base_url = 'http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas'
frutas_hortalizas_endpoint = "/ResultadosConsultaFechaFrutasYHortalizas.aspx"


with urllib.request.urlopen(mapa_precios_url) as response, \
  open('mapa.aspx', 'wb') as out_file:
    data = response.read() #.decode('utf-8').encode('utf-8') # a `bytes` object
    out_file.write(data)
    soup_directorio = BeautifulSoup(data)

    for anchor in soup_directorio.findAll('a', string=re.compile("Precio\sde\s[^la|los|Granos]\w+")):
        producto_id = re.search(r"ProductoId=(\d+)", anchor['href']).group(1)

        '''
        # Prueba descargando por producto
        if anchor.string == 'Precio de Acelga':
          flag = True
        else:
          flag = False
        if flag:
        '''
        # Si el periodo de tiempo es mayor a 5 años, se descarga por segmentos de 5 años para prevenir error en el servidor de datos
        if(dt_fecha_final.year - dt_fecha_inicio.year) > max_intervalo_años:
          for anio_inicio in range(dt_fecha_inicio.year, dt_fecha_final.year, max_intervalo_años):
            # Si el año es mayor que el año de la fecha final, se toma el de la fecha final
            if(anio_inicio + (max_intervalo_años-1)) > dt_fecha_final.year:
              anio_final = dt_fecha_final.year
            else:
              anio_final = anio_inicio + (max_intervalo_años-1)

            url = base_url + frutas_hortalizas_endpoint + f"?ProductoId={producto_id}&fechaInicio=01/01/{anio_inicio}&fechaFinal=31/12/{anio_final}&RegistrosPorPagina="
            #print(url)
            if(url.find("http:") >= 0):
              creaTabla(url,anchor.text.replace(" ","_")+"_"+str(anio_inicio)+"-"+str(anio_final), 1000, anchor.text.replace(" ","_"))
        # De lo contrario, si el periodo de tiempo es menor a 5 años, se descarga en un solo segmento
        else:
            url = base_url + frutas_hortalizas_endpoint + f"?ProductoId={producto_id}&fechaInicio={str_fecha_inicio}&fechaFinal={str_fecha_final}&RegistrosPorPagina="
            #print(url)
            if(url.find("http:") >= 0):
              creaTabla(url,anchor.text.replace(" ","_")+"_"+str(dt_fecha_inicio.year)+"-"+str(dt_fecha_final.year), 1000, anchor.text.replace(" ","_"))


http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=31&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Acelga_2020-2023 1000 Precio_de_Acelga
Página  1 de  17 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=31&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Acelga_2020-2023 17000 Precio_de_Acelga
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=130&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Aguacate_Criollo_2020-2023 1000 Precio_de_Aguacate_Criollo
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/Resultados

Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=181&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Cacahuate_2020-2023 1000 Precio_de_Cacahuate
Página  1 de  19 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=181&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Cacahuate_2020-2023 19000 Precio_de_Cacahuate
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=166&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Calabacita_Criolla_2020-2023 1000 Precio_de_Calabacita_Criolla
Página  1 de  12 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacional

Página  1 de  32 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=208&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Cilantro_2020-2023 32000 Precio_de_Cilantro
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=195&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Ciruela_Allende_2020-2023 1000 Precio_de_Ciruela_Allende
La página no tiene tabla,  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=195&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= 1000
Error:  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortali

Página  1 de  19 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=257&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Col_mediana_2020-2023 19000 Precio_de_Col_mediana
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=265&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Col_sin_clasificación_2020-2023 1000 Precio_de_Col_sin_clasificación
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=263&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Coliflor_grande_2020-2023 1000 Precio_de_Coliflor_grande
Página  1 de  14 True
http://www.economia-sniim.gob.mx/Nuevo/Cons

Página  1 de  26 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=216&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Chile_ancho_2020-2023 26000 Precio_de_Chile_ancho
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=219&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Chile_Bola_2020-2023 1000 Precio_de_Chile_Bola
La página no tiene tabla,  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=219&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= 1000
Error:  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.

...

La página no tiene tabla,  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=638&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= 1000
Error:  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=638&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= [WinError 32] The process cannot access the file because it is being used by another process: 'raw_data/Precio_de_Pera_Lechera_2020-2023.csv'
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=639&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Pera_Mantequilla_2020-2023 1000 Precio_de_Pera_Mantequilla
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNaciona

Página  1 de  27 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=723&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Plátano_Macho_2020-2023 27000 Precio_de_Plátano_Macho
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=728&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Plátano_Manzano_2020-2023 1000 Precio_de_Plátano_Manzano
Página  1 de  8 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=728&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Plátano_Manzano_2020-2023 8000 Precio_de_Plátano_Manzano
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/Mer

Página  1 de  16 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=800&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Tamarindo_2020-2023 16000 Precio_de_Tamarindo
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=804&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Tejocote_2020-2023 1000 Precio_de_Tejocote
Página  1 de  4 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=804&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Tejocote_2020-2023 4000 Precio_de_Tejocote
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agr

Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=855&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Uva_Cornichon_2020-2023 1000 Precio_de_Uva_Cornichon
La página no tiene tabla,  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=855&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= 1000
Error:  http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=855&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= [WinError 32] The process cannot access the file because it is being used by another process: 'raw_data/Precio_de_Uva_Cornichon_2020-2023.csv'
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/P

Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=871&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Uva_Superior_2020-2023 1000 Precio_de_Uva_Superior
Página  1 de  6 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=871&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Uva_Superior_2020-2023 6000 Precio_de_Uva_Superior
Página  1 de  1 False
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas/ResultadosConsultaFechaFrutasYHortalizas.aspx?ProductoId=872&fechaInicio=01/01/2020&fechaFinal=30/12/2023&RegistrosPorPagina= Precio_de_Uva_Thompson_2020-2023 1000 Precio_de_Uva_Thompson
Página  1 de  5 True
http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/